In [1]:
import os

# 딥러닝을 구동하는 데 필요한 케라스 함수 호출
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 필요한 라이브러리 불러옴
import numpy as np
import tensorflow as tf
print(tf.__version__)

import pandas as pd

# 실행할 때마다 같은 결과를 출력하기 위해 설정하는 부분
np.random.seed(3)
tf.random.set_seed(3)

# 준비된 수술 환자 데이터를 불러옴
Data_set = np.loadtxt("../dataset/ThoraricSurgery.csv", delimiter=',')

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장
X = Data_set[:, 0:17]
Y = Data_set[:, 17]


2.1.0


In [2]:

# 딥러닝 구조를 결정(모델을 설정하고 실행하는 부분)
model = Sequential()
model.add(Dense(30, input_dim=17, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

'''
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
'''

# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# 열 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=10)

# 딥러닝 실행
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, Y, epochs=100, batch_size=64,
         callbacks=[cp_callback])

model.summary()

model.save_weights('./checkpoints/my_checkpoint') # 수동으로 가중치 저장하기
model.save('./saved_model') # 전체 모델 저장하기

Train on 470 samples
Epoch 1/100
470/470 [==============================] - 1s 1ms/sample - loss: 1.3713 - accuracy: 0.7787
Epoch 2/100
470/470 [==============================] - 0s 55us/sample - loss: 0.5826 - accuracy: 0.7851
Epoch 3/100
470/470 [==============================] - 0s 51us/sample - loss: 0.5220 - accuracy: 0.7745
Epoch 4/100
470/470 [==============================] - 0s 51us/sample - loss: 0.4961 - accuracy: 0.8511
Epoch 5/100
470/470 [==============================] - 0s 57us/sample - loss: 0.4561 - accuracy: 0.8489
Epoch 6/100
470/470 [==============================] - 0s 53us/sample - loss: 0.4470 - accuracy: 0.8489
Epoch 7/100
470/470 [==============================] - 0s 53us/sample - loss: 0.4411 - accuracy: 0.8511
Epoch 8/100
470/470 [==============================] - 0s 49us/sample - loss: 0.4384 - accuracy: 0.8489
Epoch 9/100
470/470 [==============================] - 0s 57us/sample - loss: 0.4322 - accuracy: 0.8532
Epoch 10/100
 64/470 [===>..................

In [3]:
'''모델 weight 불러오기'''
checkpoint_path = "training_2/cp-0100.ckpt"
# checkpoint_path = './checkpoints/my_checkpoint'
model.load_weights(checkpoint_path) # 

loss, acc = model.evaluate(X, Y, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

print(model.predict(X).shape)

470/470 - 0s - loss: 0.3890 - accuracy: 0.8553
복원된 모델의 정확도: 85.53%
(470, 1)


In [4]:
'''모델 전체 불러오기'''
new_model = tf.keras.models.load_model('./saved_model') # 전체 모델 불러오기

# 모델 구조를 확인합니다
new_model.summary()

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(X,  Y, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(X).shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                540       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________
470/470 - 0s - loss: 0.3890 - accuracy: 0.8553
복원된 모델의 정확도: 85.53%
(470, 1)
